In [1]:
using DataFrames;
using Distributions;

In [2]:
df = readtable("/Users/nwelch/prelim/data/infectionDataTest.csv");
dst = readtable("/Users/nwelch/prelim/data/infectedDistancesTest.csv");

# Initial Parameter Values

In [92]:
#priors
mu_a=0.7; mu_b=0.004;
theta_a=0.8; theta_b=10;
sigma_a=0.5; sigma_b=100;

# initial/test values
u = mu_a*mu_b
th = theta_a*theta_b;
s = sigma_a*sigma_b;
t = [14., 10., 6.];
tl = maximum(t);

# Function Definitions

## log  Lambda

In [93]:
function getLogLambda(tau, theta, mu, sigma, dst, Tlast=30)
    iLessTlast = find(x-> x<Tlast, tau)
    gaussian = Normal(mu, sigma)
    
    sumLog = 0
    for i = iLessTlast
        sumTauJLessTauI = 0
        jLessI = find(x-> x < tau[i], tau)

        for j = jLessI
            sumTauJLessTauI = sumTauJLessTauI + pdf(gaussian, dst[i,j])
        end
        sumLog = sumLog + log(mu + theta*sumTauJLessTauI)
    end
    sumLog
end

getLogLambda (generic function with 2 methods)

In [94]:
getLogLambda(t, th, u, s, dst, tl)

-8.586772098851577

## $\mu$ log-likelihood Ratio

In [95]:
muPrior = Gamma(mu_a, mu_b)

function llRatio_mu(mu, muStar, theta, sigma, tau, dst, Tlast=30, mu_a=0.7, mu_b=0.004)
    tauLessTIndex = find(x-> x<Tlast, tau)
    
    dmu = mu - muStar
    
    dllTerm1 = sum(tau[tauLessTIndex]*dmu)
    dllTerm2 = length(tauLessTIndex) * Tlast * dmu
    
    dllTerm3a = getLogLambda(tau, theta, mu, sigma, dst, Tlast)
    dllTerm3b = getLogLambda(tau, theta, muStar, sigma, dst, Tlast)
    dllTerm3 = dllTerm3a - dllTerm3b
    
    prior = pdf(muPrior, mu)
    priorStar = pdf(muPrior, muStar)
    
    dll = dllTerm1 + dllTerm2 - dllTerm3 + prior - priorStar
    dll
end

llRatio_mu (generic function with 4 methods)

In [96]:
llRatio_mu(u, 0.01, th, s, t, dst, tl)

95.49074474191153

## $\sigma$ log-likelihood Ratio

* check the  $\le$ for Tlast in paper
* check the sign on the priors

In [97]:
function getSumLessT_sigma(sigma, sigmaStar, mu, tau, theta, dst, Tlast=30.)
    tauLessTIndex = find(x -> x<=Tlast, tau) 
    gaussian = Normal(mu, sigma)
    gaussianStar = Normal(mu, sigmaStar)
    
    sumLessT = 0
    for i = tauLessTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<=tau[i], tau)
        for j=tauJLessTauIIndex
            sum_ij = sum_ij +
                (tau[i] - tau[j])*(pdf(gaussian, dst[i,j]) -
                                   pdf(gaussianStar, dst[i,j])) 
        end
        sumLessT = sumLessT + theta*sum_ij
    end
    sumLessT
end

getSumLessT_sigma (generic function with 2 methods)

In [98]:
getSumLessT_sigma(s, 100., u, t, th, dst, tl)

0.510535174711277

In [99]:
function getSumGreaterT_sigma(sigma, sigmaStar, mu, tau, theta, dst, Tlast=30.)
    tauLessTIndex = find(x -> x<=Tlast, tau)
    tauGreaterTIndex = find(x -> x>Tlast, tau)
    gaussian = Normal(mu, sigma)
    gaussianStar = Normal(mu, sigmaStar)
    
    sumGreaterT = 0
    for i=tauGreaterTIndex
        sum_ij = 0
        for j=tauLessTIndex
            sum_ij = sum_ij + 
                (Tlast - tau[j])*(pdf(gaussian, dst[i,j]) -
                                  pdf(gaussianStar, dst[i,j]))
        end
        sumGreaterT = sumGreaterT + theta*sum_ij
    end
    sumGreaterT
end

getSumGreaterT_sigma (generic function with 2 methods)

In [100]:
getSumGreaterT_sigma(s, 100., u, t, th, dst, tl)

0

In [101]:
sigmaPrior = Gamma(sigma_a, sigma_b)

function llRatio_sigma(sigma, sigmaStar, mu, tau, theta, dst, Tlast=30.,
        sigma_a=0.5, sigma_b=100.)
    
    dllTerm1 = getSumLessT_sigma(sigma, sigmaStar, mu, tau, theta, dst)

    dllTerm2 = getSumGreaterT_sigma(sigma, sigmaStar, mu, tau, theta, dst)
    
    dllTerm3 = getLogLambda(tau, theta, mu, sigmaStar, dst, Tlast) -
        getLogLambda(tau, theta, mu, sigma, dst, Tlast)
    
    priorStar = pdf(sigmaPrior, sigmaStar)
    prior = pdf(sigmaPrior, sigma)
    
    dll = dllTerm1 + dllTerm2 + dllTerm3 + priorStar - prior
    
    dll
end

llRatio_sigma (generic function with 4 methods)

In [102]:
llRatio_sigma(s, 100., u, t, th, dst, tl)

-0.14417622119062637

# $\theta$ log-likelihood Ratio

In [103]:
function getSumLessT_theta(theta, thetaStar, mu, tau, sigma, dst, Tlast=30.)
    gaussian = Normal(mu, sigma)
    
    tauLessTIndex = find(x -> x<=Tlast, tau)
    
    sumLessT = 0
    for i = tauLessTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<=tau[i], tau)
        for j = tauJLessTauIIndex
            sum_ij = sum_ij + 
                (tau[i] - tau[j])*(theta - thetaStar)*
                    pdf(gaussian, dst[i,j])
        end
        sumLessT = sumLessT + theta*sum_ij
    end
    sumLessT
end

getSumLessT_theta (generic function with 2 methods)

In [104]:
getSumLessT_theta(th, 10., u, t, s, dst, tl)

-2.042330886608999

In [106]:
function getSumGreaterT_theta(theta, thetaStar, mu, tau, sigma, dst, Tlast=30.)
    tauLessTIndex = find(x -> x<=Tlast, tau)
    tauGreaterTIndex = find(x -> x>Tlast, tau)
    
    gaussian = Normal(mu, sigma)
    
    sumGreaterT = 0
    for i=tauGreaterTIndex
        sum_ij = 0
        for j=tauLessTIndex
            sum_ij = sum_ij + 
                (Tlast - tau[j])*(theta - thetaStar)*
                    pdf(gaussian, dst[i,j])
        end
    sumGreaterT = sumGreaterT + theta*sum_ij
    end
    sumGreaterT
end

getSumGreaterT_theta (generic function with 2 methods)

In [107]:
getSumGreaterT_theta(th, 10., u, t, s, dst, tl)

0

In [25]:
thetaPrior = Gamma(theta_a, theta_b)

function llRatio_theta(theta, thetaStar, mu, tau, sigma, dst, 
                         Tlast=30, theta_a=0.8, theta_b=10)
    
    dllTerm1 = getSumLessT_theta(theta, thetaStar, mu, tau, sigma, dst, Tlast)
    
    dllTerm2 = getSumGreaterT_theta(theta, thetaStar, mu, tau, sigma, dst, Tlast)
    
    dllTerm3 = getLogLambda(tau, thetaStar, mu, sigma, dst, Tlast) - 
                    getLogLambda(tau, theta, mu, sigma, dst, Tlast)
      
    priorStar = pdf(thetaPrior, thetaStar)
    prior = pdf(thetaPrior, theta)
    
    dll = dllTerm1 + dllTerm2 + dllTerm3 + priorStar - prior
    
    dll
end

llRatio_theta (generic function with 4 methods)

In [26]:
llRatio_theta(th, 20., u, t, s, dst, tl)

-11.393468994382678

# $\tau$ log-likelihood Ratio

In [108]:
function getMinSum(iStar, tauiStar, mu, tau, theta, sigma, dst)
    gaussian = Normal(mu, sigma)
    ti = tau[iStar]
    minTau = min(tauiStar, ti)

    jLessMin = find(x -> x<=minTau, tau)
    minSum = 0
    for j = jLessMin
        minSum = minSum + pdf(gaussian, dst[iStar,j])
    end
    minSum = (ti - tauiStar)*theta*minSum
    minSum
end

getMinSum (generic function with 1 method)

In [109]:
getMinSum(1, 12.5, u, t, th, s, dst)

0.19146852061959363

In [110]:
function getMaxSum(iStar, tauiStar, mu, tau, theta, sigma, dst)
    gaussian = Normal(mu, sigma)
    ti = tau[iStar]
    maxTau = max(tauiStar, ti)
    
    jGreaterMax = find(x -> x>maxTau, tau)
    maxSum = 0
    for j = jGreaterMax
        maxSum = maxSum + pdf(gaussian, dst[iStar,j])
    end
    maxSum = (tauiStar - ti)*theta*maxSum
    maxSum
end

getMaxSum (generic function with 1 method)

In [111]:
getMaxSum(1, 12.5, u, t, th, s, dst)

-0.0

In [112]:
function getSumIntervalBelowTauStar(iStar, tauiStar, mu, tau, theta, sigma, dst)
    
    gaussian = Normal(mu, sigma)
    ti = tau[iStar]
    
    jInBetween = find(x -> ((x>ti) & (x<tauiStar)), tau)
    sumBelow = 0
    for j = jInBetween
        sumBelow = sumBelow +
            (2*tau[j] - ti - tauiStar)*pdf(gaussian, dst[iStar,j])
    end
  sumBelow = theta*sumBelow
  sumBelow
end

getSumIntervalBelowTauStar (generic function with 1 method)

In [113]:
getSumIntervalBelowTauStar(1, 12.5, u, t, th, s, dst)

0.0

In [114]:
function getSumIntervalAboveTauStar(iStar, tauiStar, mu, tau, theta, sigma, dst)
    gaussian = Normal(mu, sigma)
    ti = tau[iStar]
    
    jInBetween = find(x -> ((x>tauiStar) & (x<ti)), tau)
    sumAbove = 0
    for j = jInBetween
            sumAbove = sumAbove +
                (ti + tauiStar - 2*tau[j])*pdf(gaussian, dst[iStar,j])
    end
    sumAbove = theta*sumAbove
    sumAbove
end

getSumIntervalAboveTauStar (generic function with 1 method)

In [115]:
getSumIntervalAboveTauStar(1, 12.5, u, t, th, s, dst)

0.0

In [123]:
function llRatio_tau(tau, tauiStar, iStar, mu, theta, sigma, dst, Tlast=30.)
    N = length(tau)
    
    dllTerm1 = mu*(tau[iStar] - tauiStar)
    dllTerm2 = getMinSum(iStar, tauiStar, mu, tau, theta, sigma, dst)
    dllTerm3 = getMaxSum(iStar, tauiStar, mu, tau, theta, sigma, dst)
    dllTerm4 = getSumIntervalBelowTauStar(iStar, tauiStar, mu, tau, 
                                          theta, sigma, dst)
    dllTerm5 = getSumIntervalAboveTauStar(iStar, tauiStar, mu, tau, 
                                          theta, sigma, dst)
    dllTerm6 = getLogLambda(tau, theta, mu, sigma, dst, Tlast)
    
    tauStar = tau
    tauStar[iStar] = tauiStar
    
    dllTerm7 = getLogLambda(tauStar, theta, mu, sigma, dst, Tlast)
    
    dll = dllTerm1 + dllTerm2 + dllTerm3 + dllTerm4 + dllTerm5 +
        dllTerm6 - dllTerm7
    dll
end

llRatio_tau (generic function with 2 methods)

In [124]:
llRatio_tau(t, 12.5, 1, u, th, s, dst, tl)

2.232466901537247